In [2]:
import torch  
import os
import torch.nn as nn 

import torch.nn.functional as F 

import torch.optim as optim 

from torch.utils.data import DataLoader 

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from PIL import Image
from torch.utils.data import Dataset, DataLoader
class SIGNSDataset(Dataset):
    def __init__(self, data_dir, transform):      
        self.filenames, self.labels = self.get_filenames(data_dir)
        image = Image.open(self.filenames[0])
        
        self.transform = transform

    def __len__(self):
        return len(self.filenames)
    
    def get_filenames(self, data_dir):
        filenames = []
        labels = []
        for class_folder in os.listdir(data_dir):
            class_path = os.path.join(data_dir, class_folder)
            folders_in_class = os.listdir(class_path)
            
            for filename in folders_in_class:
                labels.append(int(class_folder))
                path = os.path.join(class_path, filename)
                filenames.append(path)
        labels = torch.tensor(labels)
        return filenames, labels
            

    def __getitem__(self, idx):
        #open image, apply transforms and return with label
        image = Image.open(self.filenames[idx])
        
#         print(self.filenames[idx])
        image = self.transform(image)
    
        return image, self.labels[idx]

class NN(nn.Module):
    def __init__(self, input_size, num_classes): # constructor of NN with its attributes
        super(NN, self).__init__() # calling constructor of base class
        self.fc1 = nn.Linear(input_size, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 36)
        self.fc4 = nn.Linear(36,10)

    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x))
        #softmax automatically applied here
        x = self.fc4(x)
        return x


train_transformer = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
              
batch_size = 64
train_data_path = "SignLanguage\\Dataset"
train_loader = DataLoader(dataset = SIGNSDataset(train_data_path, train_transformer), 
                   batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# input_size = len(train_loader.dataset.filenames)
input_size = 3*64*64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

model = NN(input_size=input_size, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)

        targets = targets.to(device=device)
        # The enumerate() method adds a counter to an iterable
        #and returns it (the enumerate object)

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Get to correct shape, 1x28x28->784
        # -1 will flatten all outer dimensions into one
        #print (data.shape) # [64, 1, 28, 28]
        #print (targets.shape) # 64 scalar values.
        data = data.reshape(data.shape[0], -1) #[64,1x28x28]=[64, 784]
        #print (data.shape) #[64,784]

        # forward propagation
        scores = model(data) #automatically call the forward method,
                                #as model is a callable object
        loss = criterion(scores, targets) # compute cost/loss on 64 example
        # zero previous gradients
        optimizer.zero_grad()

        # back-propagation
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

In [3]:
model.train()

NN(
  (fc1): Linear(in_features=12288, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=36, bias=True)
  (fc4): Linear(in_features=36, out_features=10, bias=True)
)

In [4]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval() # 1. our model deactivates all the layers (eg.batch normalization/dropout)
    with torch.no_grad(): #2.  not make computational graph
        for x, y in loader:
            #print (x.shape)
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            #print (y.shape)
            
            scores = model(x)
             
            _, predictions = scores.max(1) #. it return max value and its index, 1 mean see column-wise 
            
#             print(predictions)
#             print(y)
            num_correct += (predictions == y).sum() # compare prediction with y, if equal sum them to count the number of same values
            num_samples += predictions.size(0)  #64, get no of samples
    print(f"Got {num_correct} / {num_samples} with accuracy"
      f" {float(num_correct) / float(num_samples) * 100:.2f}"
            )



In [5]:
model.train()
print ("Train accuracy: ")
check_accuracy(train_loader, model)

Train accuracy: 
Got 1928 / 2062 with accuracy 93.50
